In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [79]:
import pandas as pd
import pickle
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate

In [61]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/df_out.csv') # я просто не особо ещё знаю способы, как в коллабе смотреть файлы
df.head()

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
0,7308295377,https://chattanooga.craigslist.org/ctd/d/chatt...,chattanooga,https://chattanooga.craigslist.org,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,https://images.craigslist.org/00N0N_1xMPvfxRAI...,Carvana is the safer way to buy a car During t...,tn,35.060000,-85.250000,2021-04-17T12:30:50-0400,high,2021-04-17 16:30:50+00:00
1,7316380095,https://newjersey.craigslist.org/ctd/d/carlsta...,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,NaN,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnL...,***Call Us for more information at: 201-635-14...,nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
2,7313733749,https://reno.craigslist.org/ctd/d/atlanta-2017...,reno / tahoe,https://reno.craigslist.org,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,https://images.craigslist.org/00y0y_eeZjWeiSfb...,Carvana is the safer way to buy a car During t...,ca,33.779214,-84.411811,2021-04-28T03:52:20-0700,high,2021-04-28 10:52:20+00:00
3,7308210929,https://fayetteville.craigslist.org/ctd/d/rale...,fayetteville,https://fayetteville.craigslist.org,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,https://images.craigslist.org/00606_iGe5iXidib...,2013 Toyota RAV4 XLE 4dr SUV Offered by: R...,nc,35.715954,-78.655304,2021-04-17T10:08:57-0400,medium,2021-04-17 14:08:57+00:00
4,7316474668,https://newyork.craigslist.org/lgi/cto/d/baldw...,new york city,https://newyork.craigslist.org,21800,2021.0,nissan,altima,gas,8000,clean,automatic,https://images.craigslist.org/00V0V_3pSOiPZ3Sd...,2021 Nissan Altima Sv with Only 8 K Miles Titl...,ny,40.654800,-73.609700,2021-05-03T18:32:06-0400,medium,2021-05-03 22:32:06+00:00


# Data Preparation


## Data cleaning


In [62]:
numerical_cols = ['price', 'odometer', 'year']
df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].median())

categorical_cols = ['region', 'manufacturer', 'model', 'fuel', 'transmission']
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])




# Future Engineering

In [91]:
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded_features = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(categorical_cols))

df_combined = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)


# Modeling


In [92]:
X = df_combined[numerical_cols + list(encoded_df.columns)]
y = df['price_category']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [71]:
logreg= LogisticRegression(random_state=42, solver='saga')
tree = RandomForestClassifier(random_state=42)
mlp = MLPClassifier(random_state=42)

In [72]:
for model in [logreg, tree, mlp]:# Как я понимаю, логическая регрессия выдаёт ошибки, я бы могу усложнить код, убрав или доработав обработку категориальных столбцов(я пытался, но убил на это пару часов), но на это нужно много времени, да и случайный лес почти идеально высчитывает
  cv_score = cross_val_score(model, X, y, cv=5)
  print(cv_score.mean(), cv_score.std() )#


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.7565000000000001 0.06630610831590103
0.9976 0.0015297058540778572
0.8615999999999999 0.007748548251124227


## Results

In [73]:
# По результатам моделирования лучше всего себя показала модель случайного леса

In [74]:
tree.fit(X , y)

RandomForestClassifier(random_state=42)

In [86]:
filename ='model.pickle'

with open(filename, 'wb') as file:
    pickle.dump(tree, file)

In [87]:
with open(filename,'rb') as file:
  model_from_pickle = pickle.load(file)

In [88]:
model_from_pickle

RandomForestClassifier(random_state=42)

In [95]:
row =df_combined.sample(1)
row_x = row[numerical_cols + list(encoded_df.columns)]
row_y = row['price_category']

In [96]:
model_from_pickle.predict(row_x)

array(['high'], dtype=object)

In [97]:
row_y

,price_category
9759,high
